<a href="https://colab.research.google.com/github/capatommy/CDMO_Project_CORICELLI/blob/main/YT_HF_Instructor_Embeddings_Chroma_DB_Multi_Doc_Retriever_LangChain_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.276
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB - with more meta data?
- Source info
- gpt-3.5-turbo API
- HuggingFace Embeddings
- Instuctor Embeddings


## Setting up LangChain


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## Load multiple and process documents

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

def download_page(url, folder):
    response = requests.get(url)
    if response.status_code == 200:
        parsed_url = urlparse(url)
        filename = os.path.join(folder, f"{parsed_url.netloc}_{parsed_url.path.strip('/').replace('/', '_')}.html")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(response.text)

def scrape_website(base_url, folder):
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        links = soup.find_all("a")
        for link in links:
            href = link.get("href")
            if href and not href.startswith("http"):
                absolute_url = urljoin(base_url, href)
                download_page(absolute_url, folder)


In [ ]:
base_url = "https://www.toyota-europe.com/"
download_folder = "html_pages"
os.makedirs(download_folder, exist_ok=True)
scrape_website(base_url, download_folder)


In [ ]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 11.7 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import BSHTMLLoader
import os

folder_path = "./html_pages"
documents = []

# Ensure the folder path exists
if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Loop through each item in the folder
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        loader = BSHTMLLoader(item_path)
        data = loader.load()
        documents.append(data)

else:
    print("Invalid folder path")

flat_docs = [item[0] for item in documents]

In [ ]:
import re
for doc in flat_docs:
  doc.page_content = re.sub(r'[^\x20-\x7E]', '', doc.page_content)


In [ ]:
doc

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./Brochures/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents_pdf = loader.load()
flat_docs.extend(documents_pdf)

In [ ]:
len(flat_docs)

268

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(flat_docs)

In [ ]:
len(texts)

2430

In [ ]:
texts[3]

Document(page_content='ZeroToyota T-mateToyota bZWRCWECDakarFuel Cell TechnologyFuel Cell ApplicationsFuel Cell Business                    News                                    Careers                                    News                                    Careers                SearchClick to searchEnter search textClear search phraseCancel            Toyota Insurance Services        Toyotas flexible and easy car insurance optionsToyota Insurance Services offer a wide variety of flexible and convenient insurance products tailored to any customers needs.A portfolio of motor insurance products designed for allWe offer comprehensive insurance packages, at competitive premiums, tailored to every driving and mobility need.Reliable high-quality serviceWe ensure high-quality and reliable service for a Toyota, and guarantee that all repairs are carried out by Toyota Trained Technicians, using genuine Toyota parts.A one-stop Toyota experienceWe offer insurance solutions for all mobility 

## HF Embeddings

In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"

# hf = HuggingFaceEmbeddings(model_name=model_name)

## HF Instructor Embeddings

In [ ]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("what is a young car?")

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(page_content='customers, the Yaris combines style, compactness, practicality, and driving enjoyment. With Advanced Driver Assistance Systems installed as standard, its one of the safest compact cars around. Fuel consumption and CO2 emissions are low (3.7l/100km and 85 g/km).Car details                                                    Opens in new window    GR YarisA road-going sports car developed from Toyotas championship-winning World Rally experience. Its competition-inspired lightweight and highly rigid body is built on a dedicated new platform. Power comes from its 1.6-litre 3-cylinder turbocharged engine, matched with a GR-FOUR all-wheel drive system.Car details                                                    Opens in new window    Yaris CrossThe 2022 World Urban Car boasts a Big-Small packaging concept that maximises interior space and practicality; fourth generation hybrid electric power; the highest safety standards with Toyota Safety Sense; and a genuine Fun to 

In [ ]:
docs[3]

Document(page_content='ZeroToyota T-mateToyota bZWRCWECDakarFuel Cell TechnologyFuel Cell ApplicationsFuel Cell Business                    News                                    Careers                                    News                                    Careers                SearchClick to searchEnter search textClear search phraseCancel            Battery Electric Vehicles        Battery power for all-electric drivingToyotas battery electric vehicles (BEVs) are a core element in our carbon-neutrality strategy. As the companys line-up of zero emission vehicles expands, we will offer our customers accessible battery electric models across all market segments.                Rapid expansion of battery electric vehicles                By 2030, we will offer at least 30 BEV models and expect to sell around 3.5 million BEVs a year globally. In Europe, our BEV line-up will expand to ten models by 2025. To maximise their full carbon reduction potential, our vision for BEVs is to ens

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
retriever.search_type

'similarity'

In [ ]:
retriever.search_kwargs

{'k': 3}

## Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is an IO-aware exact attention algorithm that uses tiling to reduce the number of memory
reads/writes between GPU high bandwidth memory (HBM) and GPU on-chip SRAM. It is designed to improve the
quality of Transformers and enable them to scale to longer sequences.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware means optimizing for reading and writing to fast/slow memory. It has a long history in computer
science and been known by many names, such as the working set model, data locality, the Rooﬂine model of
arithmetic intensity, analyses of scalability, and standard textbook treatments of computer architecture.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in flash-attention is a technique used to split the input into blocks and make several passes over
input blocks to incrementally perform the softmax reduction.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that learns in a self-supervised way how to use different tools such as search
engines, calculators, and translation systems via simple API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can use search engines, calculators, and translation systems via simple API calls.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 We only need a handful of manually labeled examples for few-shot prompting.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 According to the text, dense and sparse retrievers can be used to augment a LM. Dense retrievers work with a
dense query and dense LMs while having less parameters, while sparse retrievers work with sparse bag-of-words
representations of the documents and the queries. Additionally, LMs can be augmented with reasoning strategies
and external tools to provide more relevant context.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf


In [ ]:
query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 REALM (Guu et al. ,2020) is the first method to jointly train end-to-end a retrieval system with an encoder
LM, while RAG (Lewis et al. ,2020) jointly fine-tunes the retriever with a sequence-to-sequence model.


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/ReACT.pdf


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f0cbf5433d0>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
